# Linear Combination of Unitaries

## Introduction:

Linear Combination of Unitaries (LCU) is a powerful quantum primitive frequently used in quantum computing, allowing us to operate on target qubit(s) with a desired weighted sum of unitary operators:
\begin{equation}
\sum_{i=0}^{2^n-1}|c_i|^2 U_i\ket{\psi}.
\end{equation}

The LCU is a cardinal tool as it allows us to operate on a target qubit with any linear algebraic operation, since essentially every square matrix may be written as a sum of unitaries (e.g., Pauli matrices form an orthonormal operator basis that spans the complete space of 2x2 square matrices). 

One of the most common uses of LCU is projecting a target qubit onto a subspace of choice. The Hadamard and Swap tests ([read more](https://docs.classiq.io/latest/tutorials/algorithms/swap-test/swap-test/?h=swap)) can be viewed as special cases of such LCUs of the general form $\frac{1}{2}(1+U)$. The Swap test, in particular, utilizes the projection of the target qubits onto the symmetric subspace, with anti-symmetric states being mapped to 0. A short [video](https://www.youtube.com/watch?v=FmZcj7O4U2w/) explaining the main concepts and uses of the LCU primitive could be of use to those of you who would like more details and motivation about LCU. 

The standard way of implementing the LCU consists of three main steps and a classical post-processing step. Assuming $n$ control qubits:  
1) Preparing the control qubit(s) in a superposition with complex amplitudes $c_i$:
\begin{equation}
\ket{0}\ket{t}\longrightarrow\sum_{i=0}^{2^n-1}c_i\ket{i}\ket{\psi}
\end{equation}
2) Selecting the unitaries $U_i$ operating on the target qubit(s), a selection controlled by the control qubit(s): 
\begin{equation}
\sum_{i=0}^{2^n-1}c_i\ket{i}\ket{\psi}\longrightarrow \sum_{i=0}^{2^n-1}c_i\ket{i}U_i\ket{\psi}
\end{equation}
3) An inverse preparation of the control qubits, which leads to a projection of the linear combination of unitaries onto the control qubit(s) $\ket{0}$ subspace, effectively factorizing the control qubit(s) from the LCU and acquiring real, non-negative, amplitudes $|c_i|^2$:
\begin{equation}
\sum_{i=0}^{2^n-1}c_i\ket{i}U_i\ket{\psi}\longrightarrow \ket{0}\sum_{i=0}^{2^n-1}|c_i|^2 U_i\ket{\psi}+ \sum_{j>0}^{2^n-1} \ket{j}\cdot\dots
\end{equation}
4) A classical post-processing selection step on the control qubit(s), where the state $\ket{0}$ is chosen in order to eliminate unwanted terms, so that we are left only with the desired LCUs operating on the target qubit(s): 
\begin{equation}
\ket{0}\sum_{i=0}^{2^n-1}|c_i|^2 U_i\ket{\psi}+ \sum_{j>0}^{2^n-1} \ket{j}\cdot\dots \underset{choose\ket{0}}{\longrightarrow}\sum_{i=0}^{2^n-1}|c_i|^2 U_i \ket{\psi}
\end{equation}


These steps can be implemented in an elegant way using the Classiq IDE and QMOD language. In Classiq, as in any proper development platform, it is recommended to think and design the desired code on a high functional level, in terms of blocks of functions.

<div style="text-align:center;"> 
<img src="https://i.ibb.co/n60brWD/image.png" alt="image" border="0"></a>
</div>


The above figure presents the 3 functional building blocks that are needed in order to implement an LCU of the following form with Classiq's IDE:
\begin{equation}
\frac{1}{2}\big(1 + \frac{1}{2}(QFT + QFT^\dagger)\big)=\frac{1}{2}(1+Real\{QFT\}) 
\end{equation}
Conditioned on that the control qubits are chosen to be $\ket{0}$. In the above equation $QFT$ marks the Quantum Fourier Transform ([read more](https://docs.classiq.io/latest/tutorials/functions/function-usage-examples/qft/qft/?h=qft)), and $QFT^{\dagger}$ its inverse. 
<details>

<summary>Quantum Fourier Transform </summary>

The Quantum Fourier Transform (QFT) function is the quantum analog for discrete Fourier transform. It is applied on the quantum register state vector in the following manner:
\begin{equation}
QFT\ket{j}=\frac{1}{\sqrt{2^n}}\sum_{k=0}^{2^n-1}e^{\frac{2\pi i}{2^n}jk}\ket{k}=\otimes_{t=1}^n\frac{1}{\sqrt{2}}\big(\ket{0}+e^{\frac{2\pi i}{2^{t}}j}\ket{1})
\end{equation}
Where $j$ and $k$ are the binary numbers represented by the $n$ qubits. Here is a short [video](https://www.youtube.com/watch?v=svSxHaDYHC0) explaining the transform. 
</details>



In the following [Guided Implementation](#guided-implementation) section we will implement these blocks, one function at a time, to receive the LCU stated in the equation above. You may also view the [Mathemtaical Description](#mathematical-description) section describing the mathematics of a general LCU implementation.

## Guided Implementation:

For the implementation of the LCU example, 2 control qubits and 2 target qubits are initiated in $\ket{0}$ states. The control qubits are referred to as `controller` and the target qubits are referred to as `psi` from now on. It should be noted that the `psi` states may be initiated at any arbitrary state and the LCU implementation will still hold, so we will refer to these states as $\ket{\psi}\ket{\psi}$ and substitute $\psi= 0$ at the end of the implementation, to verify the execution of the code.

The first block of our LCU example is the `controller` qubits `StatePreparation` stage, wherein they are prepared in the desired state encoded with the probabilities $[0.5,0.25,0.25,0]$ inserted in the `main` function (which will be discussed later).
This is easily implemented in Classiq's IDE with the aid of the `inplace_prepare_state` built-in function, which translates the probabilities by applying different simple gates on qubits. In our specific example, Hadamard `H` gates are applied to both qubits and sequentially `Ry` and `CX` gates, as the figure below demonstrates.

<div style="text-align:center;"> 
<img src="https://i.ibb.co/hyJ4Cvs/image.png" alt="image" border="0"></a></div>


The `StatePreparation` block's operation on the `controller` states may be formulated as follows: 
\begin{equation}
U_{prep}\ket{0}\ket{0}=\frac{1}{\sqrt{2}}\ket{0}\ket{0}+\frac{1}{\sqrt{4}}\ket{0}\ket{1}+\frac{1}{\sqrt{4}}\ket{1}\ket{0}
\end{equation}

In [ ]:
from classiq import CArray, CReal, QArray, QBit, inplace_prepare_state

@qfunc 
def control_preparation(probabilities: CArray[CReal],  bound: CReal, target: QArray[QBit]):
    inplace_prepare_state(probabilities,bound,target)


The next block, named `lcu_controllers`, is the main block in our LCU example. It is compactly implemented with Classiq's QMOD language by applying regular and inverted `qft` functions on the `psi` qubits, controlled by the `controllers`. 

The implementation is done with QMOD's [`control`](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/quantum-operators/#__tabbed_3_2) quantum operator, nested with the `qft` function (or the `invert` of it) in the following process: while for `controller` states corresponding to the binary representation of zero (i.e $\ket{0}\ket{0}$) the `psi` states remain unchanged, for `controller` states corresponding to the representation of 1 the `psi` states undergo a regular `qft` and for `controller` states corresponding to 2 the `psi` undergo an inverted `qft`, as can be seen in the following figure:

<div style="text-align:center;"> 
<img src="https://i.ibb.co/HqgCpj4/image.png" alt="image" border="0"></a>
</div>


In [ ]:
from classiq import QNum, IDENTITY, qft, invert

@qfunc
def lcu_controllers(controller: QNum, psi: QNum):
    control(
        ctrl= controller==0,
        operand= lambda: apply_to_all(IDENTITY,psi)
    )

    control(
        ctrl= controller==1,
        operand= lambda: qft(psi)
    )

    control(
        ctrl=controller==2,
        operand= lambda: invert(lambda: qft(psi))
    )

The combined implementation of the two previous blocks is performed by the `main` function, following an inversion of the earlier-prepared `controller` state, carried out in the third and final `InverseStatePreparation` block as can be observed in the following figure:



<div style="text-align:center;"> 
<img src="https://i.ibb.co/PQSXxdJ/image.png" alt="image" border="0"></a>
</div>

The `InverseStatePreparation` block is a necessary part of the LCU algorithm, since it projects the complete LCU onto the $\ket{0}\ket{0}$ `controller` subspace, effectively allowing us to factorize the `controller` states from the LCU (acting upon the `psi` states) in a classical post-proccessing step (which is *not* implemented in the code). For such cases, which are common in quantum computing, Classiq's QMOD offers the `Within-Apply` ([read more](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/within-apply/ )) statement, where all the operations applied within the `Compute` section (`Within` in QMOD) are only utilized for use by the functions inserted in `Action` (`Apply` in QMOD).

The `main` function works in the following sequence:
1. First, initializing the qubits by using `allocate` and setting the error bound and probabilities in which the `controller` states will be prepared.
2. Preparing the `controllers` in the state given by the inserted probabilities. This is executed by the `control_preparation` function called within the `Compute` section of the `Within-Apply` statement:
\begin{equation}
\ket{0}\ket{0}\ket{\psi}\ket{\psi}\underset{Prepare}{\longrightarrow}\bigg(\frac{1}{\sqrt{2}}\ket{0}\ket{0}+\frac{1}{\sqrt{4}}\ket{0}\ket{1}+\frac{1}{\sqrt{4}}\ket{1}\ket{0}\bigg)\otimes\ket{\psi}\ket{\psi}=\ket{\phi_1}
\end{equation}
3. Next, the `lcu_controllers` function within the `Action` section of the `Within-Apply` statement manipulates the `psi` states of the post-prepared states:
\begin{equation}
\ket{\phi_1}\underset{LCU\;Ctrl}{\longrightarrow}\frac{1}{\sqrt{2}}\ket{0}\ket{0}\ket{\psi}\ket{\psi}+\frac{1}{\sqrt{4}}\ket{0}\ket{1}QFT\ket{\psi}\ket{\psi}+\frac{1}{\sqrt{4}}\ket{1}\ket{0}QFT^\dagger \ket{\psi}\ket{\psi}=\ket{\phi_2}
\end{equation}
4. The `controller` states are then uncomputed by operating with $R_y^{\dagger}$, $CX$ and $H$ gates on them, under the execution of QMOD's `Within-Apply` statement that effectively projects the LCU onto the `controller` $\ket{0}\ket{0}$ subspace. This leads to the factorization of the unitaries acting upon the `psi` states from the `controller` $\ket{0}$ state:
\begin{equation}
\ket{\phi_2}\underset{IPrepare}{\longrightarrow}\ket{0}\ket{0}\bigg(\frac{1}{2}\ket{\psi}\ket{\psi}+\frac{1}{4}QFT\ket{\psi}\ket{\psi}+\frac{1}{4}QFT^\dagger \ket{\psi}\ket{\psi}\bigg)=\ket{0}\ket{0}\frac{1}{2}\bigg(1+\frac{1}{2}QFT+\frac{1}{2}QFT^\dagger\bigg)\ket{\psi}\ket{\psi}=\ket{0}\ket{0}\frac{1}{2}\bigg(1+Real\{QFT\}\bigg)\ket{\psi}\ket{\psi}=\ket{\phi_3}
\end{equation}

It should be noted that additional non-$\ket{0}\ket{0}$ `controller` terms that were not mentioned in the above equation survive the inverse preparation, which are expected to be observed in the execution of the model on Classiq's IDE. These states will be eliminated by the classical post-processing step of choosing the individual $\ket{0}\ket{0}$ `controller` state. 

In [ ]:
from classiq import QNum, allocate, within_apply

@qfunc
def main(controller: Output[QNum], psi: Output[QNum]):
    
    error_bound = 0.01
    
    allocate(2,psi)

    controller_probabilities= [0.5,0.25,0.25,0]
    allocate(2,controller)

    within_apply(
        compute=lambda: control_preparation(controller_probabilities,error_bound,controller),
        action= lambda:lcu_controllers(controller,psi)
        )
    
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

<details>

<summary>Note </summary>

In the native QMOD syntax in the IDE the following would look like:

```
qfunc main(output controller: qnum, output psi: qnum) {
  allocate<2>(psi);
  allocate<2>(controller);
  within {
    inplace_prepare_state<[0.5, 0.25, 0.25, 0], 0.01>(controller);
  } apply {
    lcu_controllers(controller, psi);
  }
}
```
which is a bit more intuitive.
</details>

Finally, one last task remains; verifying that the algorithm works properly. 
If we execute the LCU example with the built-in Classiq state vector simulator through the IDE, we should get results that are consistent with the following analytical result:
\begin{equation}
QFT\ket{0}\ket{0}=\frac{1}{2}\bigg(\ket{0}\ket{0}+\ket{0}\ket{1}+\ket{1}\ket{0}+\ket{1}\ket{0}\bigg)=QFT^{\dagger}\ket{0}\ket{0}=Real\{QFT\}\ket{0}\ket{0}
\end{equation}
\begin{equation}
\ket{\phi_3}=\ket{0}\ket{0}\frac{1}{2}\bigg(1+Real\{QFT\}\bigg)\ket{0}\ket{0}=\ket{0}\ket{0}\bigg(\frac{3}{4}\ket{0}\ket{0}+\frac{1}{4}\big(\ket{0}\ket{1}+\ket{1}\ket{0}+\ket{1}\ket{1}\big)\bigg)
\end{equation}
Now taking the initial `psi` states to be $\ket{\psi}\ket{\psi}=\ket{0}\ket{0}$. This state is of course not normalized, due to the other surviving terms of the inverse preparation of the `controller` states mentioned before. The results obtained from the execution on Classiq's state vector simulator are shown in the figure below:

<div style="text-align:center;"> 
<img src="https://i.ibb.co/2WZnHv0/image.png" alt="image" border="0"></a>
</div> 


While disregarding the state of the left-most qubit, the `aux` qubit, in every qubit configuration, we observe that the expected amplitudes were indeed obtained via the quantum simulation, along with the insignificant surviving terms eliminated in the classical post-selection discussed earlier.

## Mathematical Description:

The following mathematical description is for an implementation of a general LCU on a system of $n$ control qubits and $n$ target qubits, initiated in $\ket{0}\ket{\psi}$, where $\ket{\psi}$ may essentially be any state of choice. 

The control qubit(s) preparation unitary operator may be defined as:
\begin{equation}
U_c = \sum_{i=0}^{2^n-1} c_i\ket{i}\bra{0}+\sum_{j>0}^{2^n-1} \alpha_j\ket{f(j)}\bra{j} \;\;\;\;\;\;\;\; ; \;\;\;\;\;\;\;\; U_c^{\dagger} = \sum_{i=0}^{2^n-1} c_i^*\ket{0}\bra{i}+\sum_{j>0}^{2^n-1} \alpha_j^*\ket{j}\bra{f(j)}
\end{equation}
Where the left sum transforms the $\ket{0}$ control state into a superposition of $\ket{i}$ states, with complex coefficients $c_i$'s ($i$ denoting the number represented by the states). The right sum term doesn't contribute to the LCU method and accounts for transforming the states orthogonal to $\ket{0}$, in which $\alpha_j$ are complex coefficients and $f(j)$ some discrete function of its argument. Operating with $U_c$ on the control qubit(s) constitutes the preparation step of the control qubit(s):
\begin{equation}
\ket{\phi_1}=\big(U_c\otimes\mathbb{1}\big)\ket{0}\ket{\psi}= \sum_{i=0}^{2^n-1}c_i\ket{i}\braket{0|0}\ket{\psi}=\sum_{i=0}^{2^n-1}c_i\ket{i}\ket{\psi}
\end{equation}
This step is sequentially followed by a selection step, in which controlled unitary operations are successively applied to the target qubit(s). The operations are of the form $V_k=\ketbra{k}{k}\otimes \tilde{V}_k$, where $\tilde{V}_k$ are unitary operations of choice, and are controlled by corresponding control qubits in the following manner:
\begin{equation}
\ket{\phi_{2}}=\prod_{k=2^n-1}^{0} V_k\ket{\phi_{1}}=V_{2^n-1}\dots V_1V_0\sum_{i=0}^{2^n-1}c_i\ket{i}\ket{\psi}=V_{2^n-1}\dots V_1\bigg(\sum_{i=1}^{2^n-1}c_i\ket{i}\ket{\psi}+c_0\ket{0}\tilde{V}_0\ket{\psi}\bigg)=\,\dots\,=\sum_{i=0}^{2^n-1}c_i\ket{i}\tilde{V}_i\ket{\psi}
\end{equation}
Here the product of unitaries is effectively translated into a sum.
The control qubit is then operated on with the inverse unitary operation $U_c^\dagger$:
\begin{equation}\begin{split}
\ket{\phi_3} & =\big(U_c^{\dagger}\otimes\mathbb{1}\big)\ket{\phi_2}=\bigg(\sum_{k=0}^{2^n-1} c_k^*\ket{0}\bra{k}\otimes\mathbb{1}+\sum_{l>0}^{2^n-1} \alpha_l^*\ket{l}\bra{f(l)}\otimes\mathbb{1}\bigg)\sum_{i=0}^{2^n-1}c_i\ket{i}\tilde{V}_i\ket{\psi} \\
& =\sum_{i=0}^{2^n-1}\sum_{k=0}^{2^n-1}\delta_{k,i}c_k^*c_i \ket{0}\tilde{V}_i\ket{\psi}+\sum_{i=0}^{2^n-1}\sum_{l>0}^{2^n-1}\delta_{f(l),i}\alpha_l^*c_i\ket{l}\tilde{V}_i\ket{\psi}=\sum_{i=0}^{2^n-1}|c_i|^2 \ket{0}\tilde{V}_i\ket{\psi}+\sum_{l>0}^{2^n-1}\alpha_l^*c_{f(l)}\ket{l}\tilde{V}_{f(l)}\ket{\psi}
\end{split}\end{equation}
Thus projecting the sum of unitaries onto the $\ket{0}$ subspace, completely factorizing the control qubit(s) from the sum and transforming the complex coefficients into real, non-negative numbers. Some additional insignificant terms are also received as a by-product here.

This sets the ground for the last step; post-selecting the control qubit(s) to be $\ket{0}$ to eliminate the insignificant terms and obtain the carefully engineered LCU operating on the target qubit(s):
\begin{equation}
\big(\bra{0}\otimes\mathbb{1}\big)\ket{\phi_3}=\bigg(\sum_{i=0}^{2^n-1}|c_i|^2 \tilde{V}_i\bigg)\ket{\psi}
\end{equation}


## All Code Together:
### Python Version:

In [ ]:
from classiq import *

@qfunc 
def control_preparation(probabilities: CArray[CReal],  bound: CReal, target: QArray[QBit]):
    inplace_prepare_state(probabilities,bound,target)


@qfunc
def lcu_controllers(controller: QNum, psi: QNum):
    control(
        ctrl= controller==0,
        operand= lambda: apply_to_all(IDENTITY,psi)
    )

    control(
        ctrl= controller==1,
        operand= lambda: qft(psi)
    )

    control(
        ctrl=controller==2,
        operand= lambda: invert(lambda: qft(psi))
    )

@qfunc
def main(controller: Output[QNum], psi: Output[QNum]):
    
    error_bound = 0.01
    
    allocate(2,psi)

    controller_probabilities= [0.5,0.25,0.25,0]
    allocate(2,controller)

    within_apply(
        compute= lambda:control_preparation(controller_probabilities,error_bound,controller)
        action= lambda:lcu_controllers(controller,psi)
        )
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

write_qmod(quantum_model,'lcu')

### Native QMOD version:

```

// Linear Combination of Unitaries

qfunc lcu_controllers(controller: qnum, psi: qnum) {
  control (controller == 0) {
    apply_to_all<IDENTITY>(psi);
  }
  control (controller == 1) {
    qft(psi);
  }
  control (controller == 2) {
    invert {
      qft(psi);
    }
  }
}

qfunc main(output controller: qnum, output psi: qnum) {
  allocate<2>(psi);
  allocate<2>(controller);
  within {
    inplace_prepare_state<[0.5, 0.25, 0.25, 0], 0.01>(controller);
  } apply {
    lcu_controllers(controller, psi);
  }
}


```